# Custom Model Training

Use the Azure Machine Learning data collector to log various metrics

In [2]:
from azureml.logging import get_azureml_logger
logger = get_azureml_logger()

In [3]:
# Use Azure Machine Learning history magic to control history collection
# History is off by default, options are "on", "off", or "show"
# %azureml history on

Update the value of the `keras_backend` variable to switch between tensorflow or CNTK keras backends. Valid values: `tensorflow`, `cntk`

In [4]:
keras_backend = 'tensorflow' # valid values: cntk, tensorflow
os.environ['KERAS_BACKEND'] = keras_backend

In [5]:
import sys, os

from keras_training import train

project_folder = %pwd
print("Project folder: ", project_folder)
output_folder = os.path.join(project_folder,'outputs')
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

data_folder = os.path.join(project_folder, 'data')
if not os.path.exists(data_folder):
    print('Error. Path ', data_folder, ' do not exists.')

train_folder = os.path.join(data_folder, 'train')
if not os.path.exists(train_folder):
    print('Error. Path ', train_folder, ' do not exists.')

validation_folder = os.path.join(data_folder, 'validation')
if not os.path.exists(validation_folder):
    print('Error. Path ', validation_folder, ' do not exists.')

(history_tl, model) = train(train_folder, validation_folder, output_folder, epochs=16)

logger.log("Accuracy", history_tl.history['acc'])
logger.log("Loss", history_tl.history['loss'])
logger.log("Validation Accuracy", history_tl.history['val_acc'])
logger.log("Validation Losss", history_tl.history['val_loss'])


Using TensorFlow backend.


Project folder:  D:\fran\Work\eShoConAI\workbench
Found 72 images belonging to 4 classes.
Found 15 images belonging to 4 classes.
Epoch 1/16
31/31 [==============================] - 15s 489ms/step - loss: 3.1432 - acc: 0.7277 - val_loss: 2.1906 - val_acc: 0.8556
Epoch 2/16
31/31 [==============================] - 12s 384ms/step - loss: 2.8443 - acc: 0.8066 - val_loss: 8.5328 - val_acc: 0.4667
Epoch 3/16
31/31 [==============================] - 11s 358ms/step - loss: 2.8949 - acc: 0.7741 - val_loss: 2.1564 - val_acc: 0.8667
Epoch 4/16
31/31 [==============================] - 11s 363ms/step - loss: 1.2297 - acc: 0.8931 - val_loss: 3.1937 - val_acc: 0.7444
Epoch 5/16
31/31 [==============================] - 11s 368ms/step - loss: 0.9069 - acc: 0.9395 - val_loss: 2.1818 - val_acc: 0.8556
Epoch 6/16
31/31 [==============================] - 11s 367ms/step - loss: 1.3048 - acc: 0.9070 - val_loss: 4.0735 - val_acc: 0.6556
Epoch 7/16
31/31 [==============================] - 12s 389ms/step - los